In [80]:
import requests
import cv2
import base64
import time
from openai import OpenAI
import os
from IPython.display import display, Image, Audio, Markdown, Video, HTML
import http.client

# Top Videos in Niche

In [183]:
product = "gut cleanse health supplement"

In [184]:
def download_video(url, filename):
    # Send a HTTP request to the URL
    response = requests.get(url, stream=True)

    # Check if the request was successful
    if response.status_code == 200:
        # Open a local file with write-binary mode
        with open(filename, 'wb') as file:
            # Write the content to the file in chunks
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print("Download complete!")
    else:
        print("Failed to retrieve video")

In [185]:
def get_transcript(handle, video_id):    
    url = "https://tiktok-transcript.p.rapidapi.com/transcribe-tiktok-audio"

    payload = { "url": f"https://www.tiktok.com/@{handle}/video/{video_id}" }
    headers = {
    	"content-type": "application/x-www-form-urlencoded",
    	"X-RapidAPI-Key": "",
    	"X-RapidAPI-Host": "tiktok-transcript.p.rapidapi.com"
    }
    
    response = requests.post(url, data=payload, headers=headers)
    data = response.json()
    return data['response']['text']

In [186]:
number_of_shoppable_videos = 0
transcripts = []

url = "https://tokapi-mobile-version.p.rapidapi.com/v1/search/post"

headers = {
	"X-RapidAPI-Key": "",
	"X-RapidAPI-Host": "tokapi-mobile-version.p.rapidapi.com"
}

In [187]:
querystring = {"keyword": product,"count":"20","offset":"0","region":"US"}
response = requests.get(url, headers=headers, params=querystring)
data = response.json()
for idx, video in enumerate(data['search_item_list']):
    video_url = video['aweme_info']['video']['download_addr']['url_list'][0]
    if video['aweme_info']['anchors'] and video['aweme_info']['anchors'][0]['component_key'] == 'anchor_complex_shop':
        download_video(video_url, f"video{number_of_shoppable_videos}.mp4")
        transcript = get_transcript(video['aweme_info']['author']['unique_id'], video['aweme_info']['aweme_id'])
        transcripts.append(transcript)
        number_of_shoppable_videos += 1

Download complete!


In [188]:
querystring = {"keyword": product,"count":"20","offset":"20","region":"US"}
response = requests.get(url, headers=headers, params=querystring)
data = response.json()
for idx, video in enumerate(data['search_item_list']):
    video_url = video['aweme_info']['video']['download_addr']['url_list'][0]
    if video['aweme_info']['anchors'] and video['aweme_info']['anchors'][0]['component_key'] == 'anchor_complex_shop':
        download_video(video_url, f"video{number_of_shoppable_videos}.mp4")
        transcript = get_transcript(video['aweme_info']['author']['unique_id'], video['aweme_info']['aweme_id'])
        transcripts.append(transcript)
        number_of_shoppable_videos += 1

Download complete!
Download complete!
Download complete!


# Video Frame Analysis

In [189]:
client = OpenAI(api_key="")

In [190]:
responses = []
for i in range(number_of_shoppable_videos):
    video = cv2.VideoCapture(f"video{i}.mp4")
    
    base64Frames = []
    while video.isOpened():
        success, frame = video.read()
        if not success:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))
    
    video.release()

    PROMPT_MESSAGES = [
        {
            "role": "user",
            "content": [
                f"""
This is a promotional video for {product}. Provide a brief analysis of these points:
* Overall strategy used to promote the product
* Facial expressions and emotions
* Text/hooks used in the video
* How is the product demo’ed

This is the transcript of the product: {transcripts[i]}

Just use plain text, no markdown.
    """,
                *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::300]),
            ],
        },
    ]
    params = {
        "model": "gpt-4-turbo",
        "messages": PROMPT_MESSAGES,
        "max_tokens": 500,
    }
    
    result = client.chat.completions.create(**params)
    responses.append(result.choices[0].message.content)

In [191]:
message = ""
for idx, r in enumerate(responses):
    message += f"Video {idx + 1}:\n"
    message += r + '\n\n'

In [192]:
def display_top_3_videos():
    return HTML("""
    <style>
        .video-container {
            display: flex;
            justify-content: space-around;
        }
        video {
            width: 300px;  /* Adjust the width as needed */
            height: auto;  /* Maintain aspect ratio */
        }
    </style>
    <div class="video-container">
        <video controls>
            <source src="video1.mp4" type="video/mp4">
        </video>
        <video controls>
            <source src="video2.mp4" type="video/mp4">
        </video>
        <video controls>
            <source src="video3.mp4" type="video/mp4">
        </video>
    </div>
    """)

In [212]:
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": "Sythesize the insights below about why these promotional videos performed well. Break it down into these categories: 1) Overall what works. 2) Hooks used. 3) Product selling points that are resonating. And 4) Call to actions that are resonating. Provide key highlights and insights rather than listing what each video did and use no more than a few short bullet points for each video.\n\n" + message,
    },
]
params = {
    "model": "gpt-4-turbo",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 2000,
}

result = client.chat.completions.create(**params)

In [213]:
rubric = result.choices[0].message.content

In [194]:
display(display_top_3_videos())
display(Markdown(result.choices[0].message.content))

### Insights on Promotional Videos Performance

#### 1) Overall What Works:
- Direct engagement and addressing of viewer concerns such as authenticity and effectiveness resonate well.
- Straightforward, clear messaging with step-by-step guidance simplifies viewer understanding, appealing especially to those seeking practical solutions.
- Educational content, focusing on a structured regimen enhances long-term engagement and creates a sense of trust.
- A mix of humor and candid testimonials, especially about experiences of intensity and benefits, aid in creating relatable content, fostering trust and curiosity.

#### 2) Hooks Used:
- Alerts about counterfeit risks and emphasis on product authenticity catch the viewer's attention by tapping into concerns about quality.
- Simple instruction-based hooks ("3 Steps to cleanse your GUT and COLON") simplify the process, making the task seem feasible with minimal effort.
- Informative hooks combined with clear, easy-to-follow instructions appeal to viewers' desire for effective health management.
- Using humor and personal testimonial ("Be very careful with these!!") hooks potential users by balancing caution and intrigue.

#### 3) Product Selling Points That Are Resonating:
- Emphasis on authenticity and avoiding counterfeits highlights the product's quality, appealing to viewers' desire for genuine solutions.
- The ease of use and minimal modification to daily routines presented in straightforward, instructional formats address potential users' needs for convenient health solutions.
- Detailed step-by-step guidance on use and benefits, often supported by structured programs, reassure and attract viewers interested in comprehensive health management.
- Realistic expectations about the product effects (both challenging and beneficial), showcased through personal testimonials, help in setting credible perceptions and encourage trial.

#### 4) Call to Actions That Are Resonating:
- Urgent language encouraging immediate purchase effectively creates a sense of scarcity and need.
- Calls to follow simple steps (aided by text and minimalistic visuals) make the action steps clear and easy to perform, reducing barriers to purchase.
- Direct invitations to start using holistic regimens resonate by promoting health as an ongoing journey, implicitly encouraging product purchase and use.
- Personal endorsements paired with compelling visuals and cautionary tales effectively stimulate viewer curiosity and prompt immediate action, while setting realistic expectations of the product experience.

In [231]:
video = cv2.VideoCapture("eval_video.mp4")

base64Frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

video.release()

eval_transcript = '''No transcript'''
# eval_transcript = transcripts[2]
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            f"Based on this rubric below, give this input video a score out of 10 for each dimension. It's a promo video on TikTok so being casual or unique is a good thing - we're not evaluating professional commercials here. Rate of 4 dimensions: 1) Overall score. 2) Strength of hooks used. 3) Strength of product selling points. 4) Strength of call to action.\n\nTranscript of video: {eval_transcript}\n\nRubric (a video doesn't need all of these): {rubric}",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::300])
        ],
    },
]
params = {
    "model": "gpt-4-turbo",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 2000,
}

In [232]:
result = client.chat.completions.create(**params)
display(Markdown(result.choices[0].message.content))

Based on the visual content provided and the details shown in the images from the TikTok video, here’s a possible scoring based on the synthesized insights analysis rubric:

### Scoring:

#### 1) Overall score:
The video seems to focus on displaying the product in a visually appealing, clean, and straightforward manner. The simplicity could appeal to viewers who appreciate clarity and direct product focus. Since the distinctiveness of the content or narrative is unclear from just the images:
**Score: 6/10**

#### 2) Strength of hooks used:
The video uses a very streamlined visual of a product ("Water Away" by Mila MicAmor) which doesn't show any dynamic hooks such as problem alerts, emotive appeals, or humor based on the images alone. It's quite simplistic, possibly missing stronger attention-grabbing elements:
**Score: 4/10**

#### 3) Strength of product selling points:
The product seems to be positioned clearly showing its bottle, which may help viewers instantly recognize what is being sold. However, without additional context or insights into product benefits or its unique selling propositions presented within the video:
**Score: 5/10**

#### 4) Strength of call to action:
From the images provided, there is no visible call to action like a prompt to follow a link, use a promo code, or engagement-inviting phrases. This could be a missed opportunity in such advertisement content, especially on a platform like TikTok where actions can greatly influence engagement rates:
**Score: 3/10**

### Summary:
The video focuses on a clean and minimalist presentation which could be attractive in its own right but lacks strong hooks, detailed selling points, and an encouraging call to action based on the visible elements alone. Enhancing these areas could help make the video more compelling and effective in a promotional context on TikTok.